### PyCity School Analysis  
February 8, 2019  
Scott McEachern  
  


In [1]:
#- Load Datasets into DataFrame
import os
import pandas as pd


# Schools DataFrame
schoolPath = os.path.join(".", "Resources", "schools_complete.csv")

school_df = pd.read_csv(schoolPath)


# Student DataFrame
studentPath = os.path.join(".", "Resources", "students_complete.csv")

student_df = pd.read_csv(studentPath)

In [2]:
#-- Function for Formats
#  Following functions are used to format the numbers consistently throughout the report

def formatInt(sourceIntValue):
    ''' Formats int with comma for the thousands; example 23,000
    
    Accepts : sourceIntValue (int) value to be convertered'
    
    Return : (string) formatted with commas
    '''
    
    return f"{'{:,}'.format(sourceIntValue)}"


def formatCurrency(sourceValue):
    ''' Formats number for currency; example of $25.50
    
    Accepts : sourceValue (numeric) value to be converted
    
    Returns : (string) formatted for currency
    '''
    
    return f"${'{:,.2f}'.format(sourceValue)}"


def formatPercentage(sourceValue):
    ''' Formats number for percentage with 6 decimal places; example 8.123456
    
    Accepts : sourceValue (numeric) value to be converted
    
    Returns : (string) formatted for percentage
    '''
    
    return f"{'{:,.6f}'.format(sourceValue)}"

In [3]:
#-- District Summary
#   Generate the high level snapshot of district metrics

#- Calculate Total Schools
totalNumSchools = school_df.shape[0]


#- Calculate Total Students
totalNumStudents = student_df.shape[0]


#- Total School Budget
totalSchoolBudget = school_df["budget"].sum()


#- Average Math Score
aveMathScore = student_df["math_score"].mean()
                                

#- Average Reading Score
aveReadingScore = student_df["reading_score"].mean()


#- Overall Passing Rate
overallAveScore = (aveMathScore + aveReadingScore) / 2


#- Percentage students with passing math score 
totalStudentsPassingMath = student_df[student_df['math_score'] >= 70].shape[0]
percentStudentsPassingMath = (totalStudentsPassingMath / totalNumStudents) * 100


#- Percentage students with passing reading score
totalStudentsPassingReading = student_df[student_df['reading_score'] >= 70].shape[0]
percentStudentsPassingReading = (totalStudentsPassingReading / totalNumStudents) * 100

percentStudentsPassingReadingFormat = f"{'{:.6f}'.format(percentStudentsPassingReading)}"


#- Create Result Dataframe
districtSummaryResults = {
    'Total Schools': [totalNumSchools],
    'Total Students': [formatInt(totalNumStudents)],
    'Total Budget': [formatCurrency(totalSchoolBudget)],
    'Average Match Score': [formatPercentage(aveMathScore)],
    'Average Reading Score': [formatPercentage(aveReadingScore)],
    '% Passing Math': [formatPercentage(percentStudentsPassingMath)],
    '% Passing Reading': [formatPercentage(percentStudentsPassingReading)],
    '% Overall Passing Rate': [formatPercentage(overallAveScore)]
                }

districtSummary_df = pd.DataFrame(districtSummaryResults)

districtSummary_df.head()

,Total Schools,Total Students,Total Budget,Average Match Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.877840,74.980853,85.805463,80.431606


In [4]:
#-- School Summary
# Summarize the student data by the school; creates a DataFrame to store the information.

#- Merge into single dataframe
studentWithSchool_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])


#- Group by the school name
studentGroupBySchool_dfGroupBy = studentWithSchool_df.groupby('school_name')


#- Create Dictionary to store summary information
schoolSummaryData = {
    'School Name': [],
    'School Type': [],
    'Total Students': [],
    'Total School Budget': [],
    'Per Student Budget': [],
    'Average Math Score': [],
    'Average Reading Score': [],
    '% Passing Math': [],
    '% Passing Reading': [],
    '% Overall Passing Rate': []
    }


#- Calculate Summary for each school
#  Loop through the groupby and calculate the summary for each school
for schoolName, groupedSchool_df in studentGroupBySchool_dfGroupBy:
    
    # Set School Name
    schoolSummaryData['School Name'].append(schoolName)
    
    
    # Set School Type
    schoolSummaryData['School Type'].append(groupedSchool_df.iloc[0]['type'] )
    
    
    # Calculate Total Students
    schoolStudentTotal = groupedSchool_df.shape[0]
    schoolSummaryData['Total Students'].append(formatInt(schoolStudentTotal))

    
    # Total School Budget
    schoolBudget = groupedSchool_df.iloc[0]['budget']
    schoolSummaryData['Total School Budget'].append(formatCurrency(schoolBudget))
    

    # Average Student Budget
    perStudentBudget = (schoolBudget/schoolStudentTotal)
    schoolSummaryData['Per Student Budget'].append(formatCurrency(perStudentBudget))
    
    
    # Average Math Score
    schoolAveMathScore = groupedSchool_df['math_score'].mean()
    schoolSummaryData['Average Math Score'].append(formatPercentage(schoolAveMathScore))
    
    
    # Average Reading Score
    schoolAveReadingScore = groupedSchool_df['reading_score'].mean()
    schoolSummaryData['Average Reading Score'].append(formatPercentage(schoolAveReadingScore))                                                           

    
    # Percent Passing Math
    schoolStudentsPassingMath = groupedSchool_df[groupedSchool_df['math_score'] >= 70].shape[0]
    schoolPercentPassingMatch = (schoolStudentsPassingMath / schoolStudentTotal) * 100
    
    schoolSummaryData['% Passing Math'].append(formatPercentage(schoolPercentPassingMatch))
    
    
    # Percent Passing Reading
    schoolStudentsPassingReading = groupedSchool_df[groupedSchool_df['reading_score'] >= 70].shape[0]
    schoolPercentPassingReading = (schoolStudentsPassingReading / schoolStudentTotal) * 100
    
    schoolSummaryData['% Passing Reading'].append(formatPercentage(schoolPercentPassingReading))
    
    
    # Overall Passing Rate
    schoolOverallPassingRate = (schoolPercentPassingMatch + schoolPercentPassingReading) / 2
    
    schoolSummaryData['% Overall Passing Rate'].append(formatPercentage(schoolOverallPassingRate))
    
    
#- Create DataFrame of results
schoolSummary_df = pd.DataFrame(schoolSummaryData)


#- Set School Name as index
schoolSummary_df.set_index('School Name', inplace = True)


#- Display All Schools
schoolSummary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992


In [5]:
#-- Top Performing Schools

#- Sort by Passing Rate
topSchoolSummary_df = schoolSummary_df.sort_values(by=['% Overall Passing Rate'], ascending=False)


#- Create Dataframe contains top 5
#  Dateframe created rather than using Head functionality
top5SchoolSummary_df = topSchoolSummary_df[:5]


#- Display Top Schools
top5SchoolSummary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [6]:
#-- Bottom Performing Schools

#- Sort by Passing Rate
bottomSchoolSummary_df = schoolSummary_df.sort_values(by=['% Overall Passing Rate'], ascending=True)


#- Create Dataframe with bottom 5
bottom5SchoolSummary_df = bottomSchoolSummary_df[:5]


#- Display Bottom Schools
bottom5SchoolSummary_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [7]:
#-- Math & Reading Scores by Grade

#- Create Dictionary to story summary data
mathScoreSchoolSummaryData = {
    'School Name': [],
    '9th': [],
    '10th': [],
    '11th': [],
    '12th': []
    }

readingScoreSchoolSummaryData = {
      'School Name': [],
    '9th': [],
    '10th': [],
    '11th': [],
    '12th': []  
    }


#- Create List of Grades
grades = ['9th', '10th', '11th', '12th']


#- Loop through each school
for schoolName, groupedSchool_df in studentGroupBySchool_dfGroupBy:
    
    # Update Summary Data: School Name
    mathScoreSchoolSummaryData['School Name'].append(schoolName)
    
    readingScoreSchoolSummaryData['School Name'].append(schoolName)
    
    
    # Loop through each grade
    for grade in grades:
        
        # Get DataFrame that only contains rows for grade
        grade_df = groupedSchool_df.loc[groupedSchool_df['grade'] == grade]
        
        # Calculate Average Math Score
        gradeAveMathScore = grade_df['math_score'].mean()
        
        # Calculate Average Reading Score
        gradeAveReadingScore = grade_df['reading_score'].mean()
        
        # Update Summary Data
        mathScoreSchoolSummaryData[grade].append(formatPercentage(gradeAveMathScore))
        
        readingScoreSchoolSummaryData[grade].append(formatPercentage(gradeAveReadingScore))
        
        
#- Create DataFrame for Math Score  
mathScoreSchoolSummary_df = pd.DataFrame(mathScoreSchoolSummaryData)


#- Set Index of School Name
mathScoreSchoolSummary_df.set_index('School Name', inplace = True)


#- Display Math Score
mathScoreSchoolSummary_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [8]:
#-- Reading Score By Grade
#   Calculation done when doing the math score

#- Create DateFrame from Results
readingScoreSchoolSummary_df = pd.DataFrame(readingScoreSchoolSummaryData)


#- Set Index
readingScoreSchoolSummary_df.set_index('School Name', inplace = True)


#- Display Record Score
readingScoreSchoolSummary_df

,9th,10th,11th,12th
School Name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [16]:
#-- Generic Function to calculate summary

def calculateSummary(summaryColumn, studentWithSchool_df):
    ''' Calculates student results summary information based on the column provided and returns DataFrame
 
        Accepts : summaryColumn (str) column within the DataFrame provide that is to be summarized
                  studentWithSchool_df (pd.DataFrame) contains students that has been merged with schools
        
        Returns : pd.DataFrame  contains summary data for the column provided, with the following columns
                                    summaryColumn - column name was provided, contains unique values from dataset
                                    Average Math Score
                                    Average Reading Score
                                    % Passing Math
                                    % Passing Reading
                                    % Overall Passing Rate

    '''
    
    #- Prepare Dictionary to store summary results
    schoolSummaryData = {
        summaryColumn: [],
        "Average Math Score": [],
        "Average Reading Score": [],
        "% Passing Math": [],
        "% Passing Reading": [],
        "% Overall Passing Rate": []
        }


    #- GroupBy Summary Column
    studentWithSchoolGrouped_df = studentWithSchool_df.groupby(summaryColumn)

    
    #- Loop through each group and calculate summary data
    for groupName, grouped_df in studentWithSchoolGrouped_df:

        # School Size
        schoolSummaryData[summaryColumn].append(groupName)

        # Average Math Score
        schoolSummaryData['Average Math Score'].append(formatPercentage(grouped_df['math_score'].mean()))

        # Average Reading Score
        schoolSummaryData['Average Reading Score'].append(formatPercentage(grouped_df['reading_score'].mean()))

        # Determine Total Students
        schoolSummaryTotalStudents = grouped_df.shape[0]

        # Percent Passing Math
        schoolSummaryPassingMathTotal = grouped_df[grouped_df['math_score'] >= 70].shape[0]
        schoolSummaryPassingMath = ((schoolSummaryPassingMathTotal/schoolSummaryTotalStudents) * 100)

        schoolSummaryData['% Passing Math'].append(formatPercentage(schoolSummaryPassingMath))

        # Percent Passing Reading
        schoolSummaryPassingReadingTotal = grouped_df[grouped_df['reading_score'] >= 70].shape[0]
        schoolSummaryPassingReading =((schoolSummaryPassingReadingTotal / schoolSummaryTotalStudents) * 100)

        schoolSummaryData['% Passing Reading'].append(formatPercentage(schoolSummaryPassingReading))

        # Overall Passing Rate
        schoolSummaryPassingRate = (schoolSummaryPassingMath + schoolSummaryPassingReading) / 2

        schoolSummaryData['% Overall Passing Rate'].append(formatPercentage(schoolSummaryPassingRate))
    
    
    
    #- Create Data Frame
    schoolSummary_df = pd.DataFrame(schoolSummaryData)
    
    
    return schoolSummary_df


In [20]:
#-- Scores by School Spending

#- Define Summary Column Name
spendingCategoryColumn = 'Spending Category'


#- Calculate Per Student Budget; add new column to allow calculations
studentWithSchool_df['Per Student Budget'] = studentWithSchool_df['budget'] / studentWithSchool_df['size']


#- Prepare Spending Categories
spendingPerStudentBins = [0, 600, 625, 650, 675]

spendingPerStudentLabels = ["<$600", "$600-625", "$625-650", "$650-675"]


#- Create new column with DataFrame that stores spending categories
studentWithSchool_df[spendingCategoryColumn] = pd.cut(studentWithSchool_df['Per Student Budget'], 
                                                   spendingPerStudentBins, labels=spendingPerStudentLabels)

#- Calculate Summary
schoolSpendingSummary_df = calculateSummary(spendingCategoryColumn, studentWithSchool_df)


#- Set Index to Summary Column
schoolSpendingSummary_df.set_index(spendingCategoryColumn, inplace = True)


#- Display Results
schoolSpendingSummary_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Category,,,,,
<$600,83.362283,83.912412,93.738467,96.506335,95.122401
$600-625,83.544856,83.906996,93.868313,96.666667,95.267490
$625-650,77.469253,81.162258,68.659481,82.131155,75.395318
$650-675,77.034693,81.030323,66.340042,81.038136,73.689089


In [21]:
#- Scores By School Size

#- Define Column Name
schoolSizeColumn = 'School Size'


#- Prepare Categories
schoolSizeBins = [0, 1000, 2000, 5000]

schoolSizeLabels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


#- Create new column within Dataframe that stores school size category
studentWithSchool_df[schoolSizeColumn] = pd.cut(studentWithSchool_df['size'], schoolSizeBins, 
                                             labels=schoolSizeLabels)

#- Calculate Summary
schoolSizeSummary_df = calculateSummary(schoolSizeColumn, studentWithSchool_df)


#- Set Index
schoolSizeSummary_df.set_index(schoolSizeColumn, inplace = True)


#- Display Results
schoolSizeSummary_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


In [23]:
#- Scores By School Type
#  DataFrame already contains information on the school type; just calculate summary information


#- Calculate Summary
schoolTypeSummary_df = calculateSummary('type', studentWithSchool_df)


#- Rename Column
schoolTypeColumnRename = {"type": "School Type"}
schoolTypeSummary_df = schoolTypeSummary_df.rename(columns=schoolTypeColumnRename)


#- Set Index of DataFrame
schoolTypeSummary_df.set_index('School Type', inplace = True)


#- Display Results
schoolTypeSummary_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
